![MADE](resources/made.jpg)

# Академия MADE


# Семинар 10: GAN для генерации лиц

Сегодня мы займемся задачей генерации лиц с помощью генеративно-состязательных сетей (GAN). Мы не будем применять никаких эвристик, связанных именно с лицами, поэтому полученный в итоге пайплайн можно будет использовать для генерации объектов иной природы, подставив нужный датасет (ну и, скорее всего, подправив какие-то из гиперпараметров).

#### **План**:
1. **GAN & картинки: Deep Convolutional GAN (DCGAN)**
2. **DCGAN & BCELoss.**
3. **Гиперпараметры для GAN.**
4. **Анализ проблем и что делать дальше**

## 1. GAN & картинки

### 1.1. Tiny recap

![Overview](resources/gan_overview.png)

[отсюда](http://robocraft.ru/blog/machinelearning/3693.html)

В базовом случае ([Goodfellow et al, 2014](https://papers.nips.cc/paper/5423-generative-adversarial-nets.pdf)) схема обучения GAN такова:
- Есть две модели, генератор $G$ и дискриминатор $D$.
- Генератор: 
  - На вход получает вектор шума $z$,
  - На выходе показывает объект (например, картинку) $G(z)$.

- Дискриминатор: 
  - На вход получает либо настоящий, либо сгенерированный объект ($x$, $G(z)$),
  - На выходе отдает уверенность в том, что объект - настоящий ($D(x)$, $D(G(z))$).

Обучение генератора и дискриминатора производится отдельными шагами:
- "Ошибка дискриминатора" = **BCELoss**
- На шаге обучения генератора ошибка дискриминатора **максимизируется** (градиентный подъем),
- На шаге обучения дискриминатора ошибка дискриминатора **минимизируется** (good old градиентный спуск).

На лекциях было показано, что при такой схеме обучения должно происходить "сближение" двух распределений - "настоящего" $p_{data}(x)$ и "сгенерированного" $p_{g}(x)$ - с дивергенцией Йенсена-Шэннона в качестве критерия близости.

### 1.2. Засовываем картинки в GAN

Авторы оригинальной статьи использовали в экспериментах только полносвязные сети, в том числе и для генерации изображений (в низком разрешении).
Как мы знаем, там, где нужна работа с картинками, ~~полносвязным сетям места нет~~ обычно более эффективными оказываются сверточные сети. Одной из первых публикаций по теме использования сверточных сетей для генерации изображений с помощью GAN была статья ["Unsupervised Representation Learning With Deep Convolutional Generative Adversarial Networks"](https://arxiv.org/pdf/1511.06434.pdf). Основные тезисы:
* Не использовать слои "грубого" изменения размера карт активаций (`Pooling`, `Upsampling`); использовать сверточные слои (со `stride`> 1) и слои с транспонированными свертками,
* Использовать `BatchNorm` в обеих моделях,
* Не использовать полносвязные слои вообще,
* В генераторе использовать `ReLU` и `Tanh` (в конце),
* В дискриминаторе использовать `LeakyReLU`.

![dcgan](resources/dcgan.png)

Кроме того, один из соавторов статьи про `DCGAN` и ключевая фигура в разработке `PyTorch` Soumith Chintala выложил [свои рекомендации](https://github.com/soumith/ganhacks) по обучению GAN. Некоторые из них мы используем в своих экспериментах, а именно:
- Не использовать "смешанные" батчи (из настоящих и сгенерированных изображений), делать инференс по-отдельности,
- Использовать "soft labels" (`0+eps` вместо `0`, `1.0-eps` вместо `1`).

Далее соберем архитектуру, подобную `DCGAN`, обучим ее с небольшими изменениями и посмотрим, что из этого получится.

## 2. (Baseline) DCGAN & BCELoss

При установке `TRAIN` = `False` вместо обучения будут подгружаться веса модели и ожидаемые результаты.

In [ ]:
TRAIN = False

In [ ]:
# For colab
#!git clone https://github.com/lysukhin/MADE.git --branch 2021
#!mv MADE/seminars/seminar10_gan-2_faces/seminar .
#!rm -r MADE

Реализуем:
1. Подгрузку данных
2. Классы для дискриминатора и генератора
3. Функцию для обучения

In [ ]:
import os
import tqdm
import pickle
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

import torch
from torch import nn
from torch import optim
from torch import autograd

In [ ]:
from seminar.utils import get_device, batch_to_image

In [ ]:
DEVICE = get_device()
print(DEVICE)

### 2.1. Данные

Надо скачать [выровненные изображения лиц](https://cloud.mail.ru/public/ry5k/pQ8u9yut6) из датасета [CelebA](http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html). Затем распаковать и положить в папку `DATA_ROOT`.

*Параметры и данные из п.2.1. будем использовать во всех дальнейших экспериментах без изменений.* 

In [ ]:
# Пока ссылка на transfer.sh не сломается, можно скачать оттуда напрямую
#!curl https://transfer.sh/gimKy/img_align_celeba.zip -o img_align_celeba.zip
#!unzip img_align_celeba.zip
#!mkdir data
#!mv img_align_celeba data/
#!rm img_align_celeba.zip

In [ ]:
DATA_ROOT = "./data/img_align_celeba/"

IMAGE_SIZE = 64

BATCH_SIZE = 512
NUM_WORKERS = 8

NUM_TO_SHOW = 64

In [ ]:
from seminar.data import get_data  # goto
from seminar.utils import show_data_batch, save_data_batch

In [ ]:
dataset, dataloader = get_data(data_root=DATA_ROOT, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, image_size=IMAGE_SIZE)

In [ ]:
batch = next(iter(dataloader))

show_data_batch(batch, max_images=NUM_TO_SHOW)

### 2.2. Модели

Для состязательного обучения потребуются две модели - генератор и дискриминатор.

Зададим гиперпараметры моделей генератора и дискриминатора.
* `LATENT_DIM`: размерность скрытого пространства (*latent space*), т.е. длина вектора шума, из которого мы будем получать лица с помощью генератора,
* `IMAGE_CHANNELS`: число каналов в изображениях (мы будем использовать RGB-представление, поэтому каналов 3),
* `*_BASE_FEATURES`: параметры, определяющие характерную ширину сверточных слоев в обеих моделях.
* `*_NORMALIZATION`: флаги для выбора типа слоев нормализации данных внутри генератора и дискриминатора (`batch` или `none`).

In [ ]:
LATENT_DIM = 128
IMAGE_CHANNELS = 3

DISCRIMINATOR_BASE_FEATURES = 64
GENERATOR_BASE_FEATURES = 64

DISCRIMINATOR_NORMALIZATION = "batch"
GENERATOR_NORMALIZATION = "batch"

#### 2.2.1. Дискриминатор

Начнем с дискриминатора. Учтя рекомендациям авторов `DCGAN`, его структуру сделаем следующей:
* На вход подается изображение с заданным числом каналов (`input_channels`);
* Тело состоит из последовательных блоков вида `Conv2d - BN2d  - LeakyReLU`; на выходе - линейное преобразование посредством свертки;
* Сверточные слои сделаем со `stride=2`, т.к. мы не хотим использовать `MaxPool2d`; `kernel_size=4`, `padding=0`.

In [ ]:
from seminar.models import Discriminator  # goto

Модели рассчитаны на работу с изображениями `64x64` (размер уменьшается в 64 раза); проверим, что на выходе получается одно-единственное число (уверенность дискриминатора в том, что пример - реальный):

In [ ]:
discriminator = Discriminator(input_channels=IMAGE_CHANNELS, base_features=DISCRIMINATOR_BASE_FEATURES, normalization=DISCRIMINATOR_NORMALIZATION)
print(discriminator)

x = torch.randn(4, IMAGE_CHANNELS, IMAGE_SIZE, IMAGE_SIZE)
y = discriminator(x)
assert y.size() == (x.size(0), 1), y.size()

#### 2.2.2. Генератор

С генератором чуть сложнее: на вход он получает вектор шума `z` длины `LATENT_DIM`, но в следующем виде: `LATENT_DIM x 1 x 1`.
В отличие от дискриминатора, здесь используем транспонированные свертки, активации `ReLU` в середине и `Tanh` в самом конце.

Генератор тоже заточен под один размер изображений - `64x64`. 

In [ ]:
from seminar.models import Generator  # goto

In [ ]:
generator = Generator(input_channels=LATENT_DIM, base_features=GENERATOR_BASE_FEATURES, normalization=GENERATOR_NORMALIZATION, output_channels=IMAGE_CHANNELS)
print(generator)

x = torch.randn(4, LATENT_DIM, 1, 1)
y = generator(x)
assert y.size() == (x.size(0), IMAGE_CHANNELS, IMAGE_SIZE, IMAGE_SIZE), y.size()

Используем также функцию для ручной инициализации весов наших моделей:

In [ ]:
def weights_init(m, scale=0.02):
    if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.Linear)):
        torch.nn.init.normal_(m.weight, 0.0, scale)
    elif isinstance(m, nn.BatchNorm2d):
        torch.nn.init.normal_(m.weight, 0.0, scale)
        torch.nn.init.constant_(m.bias, 0)

### 2.3. Код для обучения

Чтобы генератор мог создавать изображения, на вход ему требуется подавать векторы шума. Напишем функцию, которая бы генерировала эти векторы.

In [ ]:
def generate_noise(batch_size=BATCH_SIZE, latent_dim=LATENT_DIM, device=DEVICE):
    """Create batch_size normally distributed (0, I) vectors of length latent_dim.

    Args:
        - batch_size: Number of vectors to sample.
        - latent_dim: Dimension of latent space (= length of noise vector).
        - device: Device to store output on.

    Returns:
        Torch.Tensor of shape (batch_size, latent_dim, 1, 1).
    """
    # YOUR CODE HERE
    noise = ...
    # END OF YOUR CODE
    return noise

z = generate_noise(BATCH_SIZE, LATENT_DIM, DEVICE)

assert z.size() == (BATCH_SIZE, LATENT_DIM, 1, 1), "Dimensions don't match"
assert z.mean().abs() < 0.01, "Mean of Z is not around zero, are you using normal distribution?"
assert (z.std() - 1).abs() < 0.01, "Std of Z is not around 1, are you using normal distribution?"
assert z.device == DEVICE, "Don't you forget to put output tensor to proper device."

print("All checks passed")

Сгенерируем фиксированный вектор шума, чтобы отслеживать качество работы генератора на нем по мере обучения:

In [ ]:
FIXED_NOISE = generate_noise(batch_size=64)

Теперь к самому важному - функции потерь для обучения.

Напомним алгоритм обучения GAN в [первозданном](https://papers.nips.cc/paper/5423-generative-adversarial-nets.pdf) виде:

![Algo](resources/gan_algo.png)

Для каждого батча из настоящих данных сначала обновляются веса дискриминатора (`k` раз, `k` >= 1), затем и веса генератора. 

Дискриминатор служит классификатором для отличения настоящих примеров (`real`, метка класса 1) от искусственных (`fake`, 0). В качестве функции потерь выступает бинарная кросс-энтропия (BCE), причем обновление весов дискриминатора происходит в направлении минимизации, а генератора - максимизации функции потерь.

Один из известных "лайфхаков" для обучения с BCE - это использовать вместо строгих 0 и 1 т.н. *smooth*-метки, как в ячейке ниже.

In [ ]:
LABEL_REAL = 0.95
LABEL_FAKE = 0.05

Теперь реализуем функции для шага обучения дискриминатора и генератора с BCE-Loss по-отдельности. Сделаем это в виде одного класса с 2 необходимыми методами, интерфейс для которого описан в классе `GANLoss`:

In [ ]:
class GANLoss(nn.Module):
    """Base class for GAN training loss functions."""
    
    def generator(self, discriminator, generator, real_batch, fake_batch, device):
        """Make 1 training step for generator.
        
        Args:
            - discriminator: Discriminator object.
            - generator: Generator object.
            - real_batch: Batch of real images (compatible with discriminator).
            - real_batch: Batch of fake images from generator (compatible with discriminator).
            - device: Device to use.
            
        Returns:
            Torch.Tensor of size 1 with resulting loss value for generator.
        """
        raise NotImplementedError

    def discriminator(self, discriminator, generator, real_batch, fake_batch, device):
        """Make 1 training step for generator.
        
        Args:
            - discriminator: Discriminator object.
            - generator: Generator object.
            - real_batch: Batch of real images (compatible with discriminator).
            - real_batch: Batch of fake images from generator (compatible with discriminator).
            - device: Device to use.
            
        Returns:
            Torch.Tensor of size 1 with resulting loss value for generator.
        """
        raise NotImplementedError

Нам нужно реализовать по этому примеру класс для работы с BCE Loss.

Для дискриминатора:
- Пропустить через дискриминатор `real_batch` / `fake_batch`, получить активации.
- Приготовить правильные метки для двух классов (используя `soft labels`, как описано выше).
- Посчитать `BCELoss` (точнее, `BCEWithLogitsLoss`, поскольку нелинейности на выходе дискриминатора нет) для двух наборов активаций и вернуть среднее.

Для генератора:
- Пропустить через дискриминатор `fake_batch`, получить активации.
- Приготовить "неправильные" метки для класса `fake` (то есть подсунуть метки класса `real`).
- Посчитать `BCEWithLogitsLoss` и вернуть его значение.

In [ ]:
from torch.nn.functional import binary_cross_entropy_with_logits

class GANBCELoss(GANLoss):
    
    def generator(self, discriminator, generator, real_batch, fake_batch, device):
        
        # YOUR CODE HERE
        prob_fake = ...
        labels_fake = ...
        generator_loss = ...
        # END OF YOUR CODE
        
        return generator_loss

    def discriminator(self, discriminator, generator, real_batch, fake_batch, device):
        
        # YOUR CODE HERE
        prob_real = ...
        prob_fake = ...
        labels_real = ...
        labels_fake = ...
        discriminator_loss = ...
        # END OF YOUR CODE
    
        return discriminator_loss

    
from seminar.losses import GANBCELoss as GANBCELossCorrect

generator.to(DEVICE)
discriminator.to(DEVICE)

bce_loss = GANBCELoss()
bce_loss_correct = GANBCELossCorrect()

real_batch = next(iter(dataloader))
fake_batch = generator(FIXED_NOISE.to(DEVICE))

assert bce_loss.generator(discriminator, generator, real_batch, fake_batch, DEVICE) == \
    bce_loss_correct.generator(discriminator, generator, real_batch, fake_batch, DEVICE), \
    "Wrong outputs for generator loss"

assert bce_loss.discriminator(discriminator, generator, real_batch, fake_batch, DEVICE) == \
    bce_loss_correct.discriminator(discriminator, generator, real_batch, fake_batch, DEVICE), \
    "Wrong outputs for discriminator loss"

print("All checks passed")

Теперь, когда у нас есть составные компоненты, используем (более-менее) универсальную обвязку для обучения в рамках 1 эпохи.

In [ ]:
from seminar.training import train_epoch  # goto

### 2.4. Обучение

Сети GAN могут быть очень чувствительны к изменениям гиперпараметров. О том, как можно упростить процесс их подбора, поговорим чуть позже, а пока используем рабочий набор гиперпараметров, с которым обучение должно быть стабильным и давать неплохой результат.

In [ ]:
LR = 0.0003
BETAS = [0.5, 0.999]
N_CRITIC = 1

NUM_EPOCHS = 64

In [ ]:
generator = Generator().to(DEVICE)
generator.apply(weights_init)
optimizer_generator = optim.Adam(generator.parameters(), lr=LR, betas=BETAS, amsgrad=True)

discriminator = Discriminator().to(DEVICE)
discriminator.apply(weights_init)
optimizer_discriminator = optim.Adam(discriminator.parameters(), lr=LR, betas=BETAS, amsgrad=True)

loss_fn = GANBCELoss()

In [ ]:
run_name = "00_bce_64x64_lr3e-4"

In [ ]:
# Пока ссылка на transfer.sh не сломается, можно скачать оттуда напрямую
# !curl https://transfer.sh/123Lp14/00_bce_64x64_lr3e-4.zip -o 00_bce_64x64_lr3e-4.zip
# !unzip 00_bce_64x64_lr3e-4.zip
# !mkdir runs 
# !mv 00_bce_64x64_lr3e-4 runs/
# !rm 00_bce_64x64_lr3e-4.zip

In [ ]:
run_dirname = os.path.join("runs", run_name)
checkpoint_filename = os.path.join(run_dirname, "checkpoint.pth.tar")
results_filename = os.path.join(run_dirname, "results.pkl")
images_dirname = os.path.join(run_dirname, "images")

if TRAIN:
    
    os.makedirs(run_dirname)
    os.makedirs(images_dirname)
    
    results = {
        key: [] for key in [
            "generator_loss_list", 
            "discriminator_loss_list", 
            "generated_batch_list"
        ]
    }
    
    for epoch in range(NUM_EPOCHS):
        epoch_results = train_epoch(generator, discriminator, 
                                    optimizer_generator, optimizer_discriminator,
                                    dataloader, epoch, NUM_EPOCHS, 
                                    LATENT_DIM, FIXED_NOISE, loss_fn, N_CRITIC, 
                                    DEVICE)
        for key in results:
            results[key].extend(epoch_results[key])
            
        with open(results_filename, "wb") as fp:
            pickle.dump(results, fp)

        with open(checkpoint_filename, "wb") as fp:
            torch.save({"generator": generator.state_dict(),
                        "discriminator": discriminator.state_dict()}, fp)
        
        image_filename = os.path.join(images_dirname, f"batch_ep={str(epoch).zfill(2)}.png")
        save_data_batch(epoch_results["generated_batch_list"][0], image_filename)

else:
    print("%3 hours later%")
    with open(results_filename, "rb") as fp:
        results = pickle.load(fp)
    with open(checkpoint_filename, "rb") as fp:
        checkpoint = torch.load(fp, map_location="cpu")
    generator.load_state_dict(checkpoint["generator"])
    discriminator.load_state_dict(checkpoint["discriminator"])

Посмотрим на графики значений лосса для обеих моделей:

In [ ]:
plt.figure(figsize=(16,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(results["generator_loss_list"], label="G", alpha=0.5)
plt.plot(results["discriminator_loss_list"], label="D", alpha=0.5)
plt.xlabel("iterations")
plt.ylabel("Loss")
# plt.yscale("log")
plt.legend()
plt.show()

Минус обучения GAN с BCE в том, что графики лосса плохо отражают суть происходящего с моделями. С одной стороны, если значения лоссов сильно отличается друг от друга (на порядок), то это может говорить об остановке обучения и т.н. коллапсе. Однако, даже если "на глаз" графики кажутся "нормальными", это не говорит о том, что генератор в самом деле выдает что-то хорошее.

Поэтому лучше было бы иметь некую дополнительную (или несколько) метрику, которая бы отражала прогресс обучения генератора. В ее качестве можно взять `Frechet Inception Distance`, о ней поговорим чуть позже.

Отрисуем сгенерированные примеры, полученные на разных эпохах обучения из вектора `FIXED_NOISE`:

In [ ]:
#%%capture
fig = plt.figure(figsize=(16,16))
plt.axis("off")
imgs = [[plt.imshow(batch_to_image(batch[:64]), animated=True)] for batch in results["generated_batch_list"][::8]]
ani = animation.ArtistAnimation(fig, imgs, interval=1000, repeat_delay=1000, blit=True)

HTML(ani.to_jshtml())

Генератор, как видно, двигался в правильном направлении - он действительно стал выдавать что-то похожее на лица. Однако количество артефактов довольно велико, и не все из результатов можно назвать "адекватными".

С помощью ячейки ниже можно погенерировать примеров и поисследовать характерные артефакты.

In [ ]:
noise = generate_noise(batch_size=64)

generator.eval()
with torch.no_grad():
    generated = generator(noise)

show_data_batch(generated)

Частые проблемы генератора:
* Нет консистентности частей лица (например, мужские лица + женские прически)
* Нереалистичкая форма лица
* Сильный шум на фоне
* Искаженные лица

Немного поиграем с получившимся генератором. Возьмем два батча векторов шума и выполним линейную интерполяцию между ними:

In [ ]:
def interpolate_noise(batch1, batch2, num):
    """Interpolate between two tensors of the same shape.

    Args:
        - noise1: First ('start') tensor (shaped b x latent_dim x *).
        - noise2: Second ('end') tensor (shaped b x latent_dim x *).
        - num: Number of points in interpolated output.

    Returns:
        List of num torch.Tensors (each shaped b x latent_dim x *).
    """
    
    step_size = 1 / (num - 1)
    weights = torch.tensor([i * step_size for i in range(num)])
    interpolation_list = [torch.lerp(noise1, noise2, w) for w in weights]
    
    return interpolation_list

In [ ]:
generator.eval()

noise1 = generate_noise(batch_size=8, device="cpu")
noise2 = generate_noise(batch_size=8, device="cpu")
interpolated = interpolate_noise(noise1, noise2, num=8)

for i in range(len(interpolated)):
    noise = interpolated[i]
    with torch.no_grad():
        generated = generator(noise.to(DEVICE))
    show_data_batch(generated)

## 3. Что дальше?

Есть несколько вариантов того, что можно сделать для улучшения результатов:
1. Более тщательно подобрать гиперпараметры в текущем подходе.
2. Использовать альтернативные функции потерь, о которых можно узнать из лекций / статей / etc (например, `Wasserstein Loss` или `Hinge Loss`).
3. Внести изменения в архитектуру моделей, направленные на улучшение качества генерации или устойчивости обучения (`Self-Attention`, `Spectral Normalization`, ...).

В любом случае, процесс улучшения GAN будет трудоемким: изменение функции потерь (п.2) или архитектуры (п.3) может потребовать заново подбирать гиперпараметры (п.1) уже настроенного пайплайна.
Поэтому уделим время тому, как это можно сделать с меньшими трудозатратами.

### 3.1. Optuna

На [одном](https://data.mail.ru/curriculum/program/lesson/16359/) из прошлых занятий обсуждался фреймворк для оптимизации гиперпараметров [Optuna](https://optuna.org/). Коротко напомним, как с ним работать:

1. Заменить явное присвоение значений интересующих нас гиперпараметров на вызовы вида `trial.suggest_int` (`_float`, `_categorical`, ...), указав области значений (нижнюю/верхнюю границы или список категорий). Optuna будет в каждом испытании (`Trial`) подставлять значение из данной области.
2. Обернуть обучение в целевую функцию и передать ее для оптимизации специальному объекту `optuna.Study`, который сам запустит указанное число испытаний и соберет итоговые результаты.

Целевая функция должна возвращать значение, которое optuna будет использовать для оценки данного набора гиперпараметров. Оно должно отражать качество работы ваших моделей; в случае с GAN, это должна быть метрика качества генерации. 

### 3.2. Frechet Inception Distance

Распространенным подходом к оценке качества генерации является `Frechet Inception Distance` (`FID`) между наборами настоящих и сгенерированных изображений.
 
* `Frechet Distance` - это показатель для сравнения двух распределений $p_1$($\mu_1$, $\Sigma_1$) и $p_2$($\mu_2$, $\Sigma_2$); вычисляется как $$fd^2 = ||\mu_1 - \mu_2||^2 + Tr(\Sigma_1 + \Sigma_2 - 2\sqrt{\Sigma_1 \Sigma_2})$$
* Чем ближе `FD` к нулю, тем более "похожи" распределения.
* В качестве случайных величин в FID используются активации слоев для feature_extraction предобученного (на ImageNet) классификатора Inception; отсюда `I` в `FID`.
* [Как показывает практика](https://arxiv.org/pdf/1801.03924.pdf), признаки из предобученных на ImageNet моделях неплохо коррелируют с человеческим восприятием "похожести", что позволяет надеяться на FID как на адекватную метрику качества генерации реалистичных изображений. 

То есть алгоритм оценки `FID` на наборе реальных данных такой:
1. Сгененировать с помощью модели генератора выборку "фейковых" данных.
2. Пропустить ее через предобученную модель Inception и сохранить активации необходимых слоев для feature_extraction.
3. Пропустить также выборку реальных данных.
4. Посчитать `FD` между распределениями активаций для двух доменов.  



Реализацию вычисления `FID` (и других метрик) можно найти, например, в [репозитории](https://github.com/toshas/torch-fidelity) `torch_fidelity`.

In [ ]:
# train-opt.py  # goto

## 4. GAN Readlist

Статьи:
1. [Generative Adversarial Networks](https://papers.nips.cc/paper/5423-generative-adversarial-nets.pdf) (GAN)
2. [Conditional GAN](https://arxiv.org/abs/1411.1784) (cGAN)
3. [Unsupervised representation learning with deep convolutional generative adversarial networks](https://arxiv.org/pdf/1511.06434) (DCGAN)
4. [Image-to-Image Translation with Conditional Adversarial Networks](https://arxiv.org/abs/1611.07004) [Pix2Pix]
5. [Wasserstein GAN](https://arxiv.org/abs/1701.07875) (Wasserstein GAN)
6. [Improved Training of Wasserstein GANs](https://arxiv.org/abs/1704.00028) (Wasserstein GAN + Gradient Penalty)
7. [Spectral Normalization for Generative Adversarial Networks](https://arxiv.org/abs/1802.05957) (SNGAN)
8. [Self-Attention Generative Adverarial Networks](https://arxiv.org/pdf/1805.08318) (SAGAN)
9. [Large Scale GAN Training for High Fidelity Natural Image Synthesis](https://arxiv.org/abs/1809.11096) (BigGAN)
10. [A Style-Based Generator Architecture for Generative Adversarial Networks](https://arxiv.org/abs/1812.04948) (StyleGAN)

Репозитории и код:
1. [PyTorch DCGAN example](https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html) (основа для этого семинара)
2. [PyTorch-GAN](https://github.com/eriklindernoren/PyTorch-GAN) (простые реализации множества статей по GAN)
3. [BigGAN](https://github.com/eriklindernoren/PyTorch-GAN) (реализация огромного числа фич для GAN)
4. [BigGAN TF Hub Demo](https://colab.research.google.com/github/tensorflow/hub/blob/master/examples/colab/biggan_generation_with_tf_hub.ipynb) (ноутбук с предобученным BigGAN)
5. [How to Train a GAN? Tips and tricks to make GANs work](https://github.com/soumith/ganhacks) (осторожно, многие советы устарели)